# Welcome to UBLyter !

_UBLyter_ is an [everis](https://www.everis.com/global/en) **SEMBU** environment for the generation of [OASIS UBL-2.x](http://docs.oasis-open.org/ubl/cs01-UBL-2.2/UBL-2.2.html) artefacts. The current version is UBLyter v0.1, which focuses only on the Pre-award stage. The status of the development is "just started!". Feel free to contribute by creating issues and commenting on the [Ublyter GitHub](https://github.com/everis-sembu/ublyter).

## Edit the model

* Download the [UBL-2.3 model](./mod/UBL-Pre-award-2.3.ods),

* Modify the model (introduce the changes you need),

* Upload it ([how to upload](http://everis-sembu.github.io/how-to-upload.html))

## Set the main distribution parameters

In [3]:
String 
    modelName      = "Pre-award",
    stage          = "preaward02", 
    version        = "test", // the name of the target resulting version
    datetimelocal  = "20190331-1500z",
    UBLversion     = "2.3"; // the version of the UBL distribution this model is based on



In [4]:
// Cranesoftwrights tooling timestamps
String 
    gc2obdndr = "20190320-0140z",
    ods2obdgc = "20180921-2010z",
    cvagcxsl  = "20130416-0040z",
    cva2sch   = "20130207-1940z",
    gc2odsxml = "20170727-0220z";

## Environment configuration

### Location of the `*.jar` files

Jupyter-notebook needs to know where to locate the saxon and other java resources. 

The code below is intended uniquely for Linux. It wont work for Windows. Check this URL for an example of how to handle this in other OS: https://stackoverflow.com/questions/318239/how-do-i-set-environment-variables-from-java

#### Set environment variable(s)

[TODO]: this is not working. Review, please.

In [5]:
import java.lang.reflect.Field;

String key = "IJAVA_CLASSPATH", value = "./lib/*.jar";

try {
        Map<String, String> env = System.getenv();
        Class<?> cl = env.getClass();
        Field field = cl.getDeclaredField("m");
        field.setAccessible(true);
        Map<String, String> writableEnv = (Map<String, String>) field.get(env);
        writableEnv.put(key, value);
    } catch (Exception e) {
        throw new IllegalStateException("Failed to set environment variable", e);
    }



#### Check that the environment variable(s) are seen by the Jupyter-notebook

In [6]:
Map<String, String> env = System.getenv();
System.out.println("IJAVA_CLASSPATH = " + env.get("IJAVA_CLASSPATH"));

IJAVA_CLASSPATH = ./lib/*.jar


### Global directory and file path names

The following variables are used along the lifecycle of the execution to specify input and output directory and file names.

In [7]:
String targetDir = "./" + stage + "-" + UBLversion + "/"; // Ouput distribution directory 
String mod       = "./mod/"; // Location of the model. In UBL models are maintained as spread-sheets!
String ident     = "./ident/"; // Metadata for the identification of the BIE(s), namespaces, UBLversion, etc.
String massage   = "./massage/"; // ODS tabs names need to be spanned
String util      = "./util/"; // Utilities directory: Cranesoftwright's XSL-T files
String lengthen_model_name_uri = massage + "massageModelName-" + UBLversion + ".xml";

#### Check variables

In [8]:
System.out.println(targetDir);
System.out.println(mod);
System.out.println(ident );
System.out.println(massage );
System.out.println(lengthen_model_name_uri);

./preaward02-2.3/
./mod/
./ident/
./massage/
./massage/massageModelName-2.3.xml


## Raw files

The UBL distribution is organised around a bunch of **constant** folders and files that are part of the data architecture, data validation, documentation, etc. There is no point in creating these from scratch every time, so they are copied from a kind of template (*raw*) directory into the target output directory.

This implies that the "raw" folder, and its content, is necessary **before** the production of the XSD schemas. So make sure that the **_raw_** folder exists and contains the right original files. 

The next cells are used to:

1. Specify where these "raw" folder and files are located, and
2. Copy them in the target (output) directory.

In [9]:
String rawDir = "./" + "raw" + "/";

In [10]:
import org.apache.commons.io.FileUtils;
import java.io.File;
import java.io.IOException;

try {
    FileUtils.copyDirectory(new File(rawDir), new File(targetDir));
} catch (IOException e) {
    e.printStackTrace();
}


## Creation of the GeneriCode (GC) files

[GeneriCode](http://docs.oasis-open.org/codelist/ns/genericode/1.0/) (GC) is an OASIS XML specification for the expression of tables. The OASIS UBL tooling uses GC as a **_pivot_** format that is used in several different ways, e.g. to check NDR and as the input for subsequent generation of W3C XSD schemas, [CVA](http://docs.oasis-open.org/codelist/cs01-ContextValueAssociation-1.0/doc/context-value-association.html) files, HTML reporting and other types of transformations.

### Configuration

#### Set file path names for the transformation

In [12]:
// Cranesoftwright's transformation stylesheet
String xsl     = "-xsl:" + util + "Crane-ods2obdgc-" + ods2obdgc + "/Crane-ods2obdgc.xsl";

// Input files for the transformation
String
    source1                 = mod + "UBL-Library-" + UBLversion + ".ods",
    source2                 = mod + "UBL-Documents-" + UBLversion + ".ods",
    identification          = ident + "ident-UBL-"+ UBLversion + "-" + modelName + ".xml",
    lengthen_model_name_uri = massage + "massageModelName-" + UBLversion + ".xml";

//Output file
String target = mod + "UBL-Entities-" + UBLversion + ".gc";

// Parameters for the Transform class
String xsl     = "-xsl:" + util + "Crane-ods2obdgc-" + ods2obdgc + "/Crane-ods2obdgc.xsl";
String o       = "-o:" + target;
String it      = "-it:ods-uri";
String ods_uri = "ods-uri=" + source1 + "," + source2;
// The XSL-T base directory is "./util/Crane-ods2obdgc-20180921-2010z" (whilst the jupyter notebook is "./") hence the ident file is in "../../ident/"
String identification_from_util_folder = "../../ident/"; 
String ident_uri = "identification-uri=" + identification_from_util_folder + "ident-UBL-"+ UBLversion + "-" + modelName + ".xml";

#### Check file path names

In [13]:
// Show the files names. For debug purposes only. Remove in final version.
System.out.println(source1);
System.out.println(source2);
System.out.println(target);
System.out.println(identification);
System.out.println(lengthen_model_name_uri);
System.out.println(target);
System.out.println(xsl);
System.out.println(o);
System.out.println(it);
System.out.println(ods_uri);
System.out.println(identification_from_util_folder);
System.out.println(ident_uri);

./mod/UBL-Library-2.3.ods
./mod/UBL-Documents-2.3.ods
./mod/UBL-Entities-2.3.gc
./ident/ident-UBL-2.3-Pre-award.xml
./massage/massageModelName-2.3.xml
./mod/UBL-Entities-2.3.gc
-xsl:./util/Crane-ods2obdgc-20180921-2010z/Crane-ods2obdgc.xsl
-o:./mod/UBL-Entities-2.3.gc
-it:ods-uri
ods-uri=./mod/UBL-Library-2.3.ods,./mod/UBL-Documents-2.3.ods
../../ident/
identification-uri=../../ident/ident-UBL-2.3-Pre-award.xml


#### Check if input files are present

(_If no error messages are thrown then all the files are present_)

In [14]:
if(!new File(source1).exists()) System.err.println("File " + source1 + " was not located in folder " + mod);
if(!new File(source2).exists()) System.err.println("File " + source2 + " was not located in folder " + mod);
if(!new File(identification).exists()) System.err.println("File " + identification + " was not located in folder " + ident);
if(!new File(lengthen_model_name_uri).exists()) System.err.println("File " + lengthen_model_name_uri + " was not located in folder " + massage);

### Generation of the GC files

In [22]:
import net.sf.saxon.Transform;
import java.text.SimpleDateFormat;
import java.util.Calendar;

Calendar cal = Calendar.getInstance();
SimpleDateFormat sdf = new SimpleDateFormat("HH:mm:ss");    

String[] arglist = {xsl, o, it, ods_uri, ident_uri};

System.out.println("Transformation started at: " + sdf.format(cal.getTime()) + "this may take some minutes...");

Transform.main(arglist);

sdf = new SimpleDateFormat("HH:mm:ss");    
System.out.println("Transformation ended at: " + sdf.format(cal.getTime());

Transformation started at: 02:10:00this may take some minutes...


EvaluationInterruptedException: Evaluator was interrupted while executing: 'Transform.main(arglist);'